# DLKit Models

DLKit is a lightweight framework for managing a large number of Keras models. It provides a wrapper for models, tools to efficiently represent and read data, and analysis functions. This notebook introduces the DLKit `ModelWrapper`.

## DLModels
To create a DLKit model, create a new class that inherits from `DLTools.ModelWrapper` and implement the constructor (`__init__`) and `Build` functions. For example:


In [ ]:
from DLTools.ModelWrapper import ModelWrapper

from keras.layers import Input, Dense
from keras.models import Model

class MLPClassification(ModelWrapper):
    def __init__(self, Name,
                 InputShape=(None,5),
                 Widths=[10],
                 Activation="relu",
                 Loss="categorical_crossentropy",
                 Optimizer="SGD"):

        super(MLPClassification, self).__init__(Name, Loss, Optimizer)

        self.InputShape = self.MetaData["InputShape"] = InputShape
        self.Widths = self.MetaData["Widths"] = Widths
        self.Activation = self.MetaData["Activation"] = Activation

    def Build(self):
        myInput = Input(shape=self.InputShape)
        myModel = myInput

        for i in range(0, len(self.Widths)-1):
            myModel = Dense(self.Widths[i], activation=self.Activation)(myModel)

        # Use softmax activation for final layer for classification
        i+=1
        myModel = Dense(self.Widths[i], activation="softmax")(myModel)

        self.Model = Model(inputs=myInput, outputs=myModel)


Note that you specify any configurable parameters of you model in the constructor. Also, you can store information in the `MetaData` dictionary. This information, along with the model architecture and weights, will be saved when `ModelWrapper.Save()` is called and restored when `ModelWrapper.Load()` is called. This information will be stored in `TrainedModels/<Name>`, where `<Name>` is the name of the model you provide when you instantiate the model. For example:

In [ ]:
# Instantiate a model which takes in 10 inputs per example, has hidden layers 
# of width 5, 10, and 20, and classifies into 4 classes.
MyModel=MLPClassification("MyFirstModel",InputShape=(None,10), Widths=[5,10,20,4] )

# Now Build it
MyModel.Build()

# And Compile
MyModel.Compile()

# Get Summary from the Keras model
MyModel.Model.summary()

MyModel.Save()

Note that `MyModel.Model` is the Keras model, which you can then use to fit or evaluate. You can see the saved model in the `TrainedModels` directory:

In [ ]:
%ls TrainedModels
%ls TrainedModels/MyFirstModel

You can load a model back:

In [ ]:
LoadedModel=ModelWrapper("MyFirstModel")
LoadedModel.Load()
print LoadedModel.MetaData

Note that by default, `Save()` does not overwrite existing models with the same name. Also note that `MetaData["InputMetaData"]` is a list of all previous saves of the model. 